# Classification that is never fair

In this notebook we try to train a fair classifier on a dataset that is inherently unfair; i.e. the
base rate varies between the protected classes. The dataset is generated in `gen-unfair-data.ipynb`.
The idea behind this notebook is that is impossible to create such a classifier which is fair in
false positives (i.e. people who get tagged but should not be) as well as false negatives (people
who should be tagged but aren't) **and** performs better than random.

In [ ]:
# HIDE
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


torch.manual_seed(1)
np.random.seed(7)
sns.set(style="white", palette="muted", color_codes=True, context="talk")

%matplotlib inline

In [ ]:
# HIDE
def load_GEN_data(path):
    input_data = pd.read_csv(path)
    
    # sensitive attributes; we identify 'race' and 'sex' as sensitive attributes
    sensitive_attribs = ['protected']
    Z = (input_data.loc[:, sensitive_attribs]
         .assign(protected=lambda df: (df['protected'] == 'B').astype(int)))

    # targets; 1 when someone makes over 50k , otherwise 0
    y = input_data['target'].astype(int)

    # features; note that the 'target' and sentive attribute columns are dropped
    X = (input_data
         .drop(columns=['target', 'protected'])
         .pipe(pd.get_dummies, drop_first=True))
    
    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape} samples")
    print(f"sensitives Z: {Z.shape[0]} samples, {Z.shape[1]} attributes")
    return X, y, Z


def p_rule(y_pred, z_values, threshold=0.5):
    y_z_1 = y_pred[z_values == 1] > threshold if threshold else y_pred[z_values == 1]
    y_z_0 = y_pred[z_values == 0] > threshold if threshold else y_pred[z_values == 0]
    odds = y_z_1.mean() / y_z_0.mean()
    return np.min([odds, 1/odds]) * 100

def fp_rate(y_test, y_pred, z_values, threshold=0.5):
    z_1 = y_pred[(y_test == False) & (z_values == 1)]
    z_0 = y_pred[(y_test == False) & (z_values == 0)]
    return {0: (z_0 > threshold).sum() / len(z_0), 1:(z_1 > threshold).sum() / len(z_1)}

def fn_rate(y_test, y_pred, z_values, threshold=0.5):
    z_1 = y_pred[(y_test == True) & (z_values == 1)]
    z_0 = y_pred[(y_test == True) & (z_values == 0)]
    return {0: (z_0 <= threshold).sum() / len(z_0), 1:(z_1 <= threshold).sum() / len(z_1)}

def plot_dists(y_true, Z_true, y_pred, Z_pred=None, epoch=None):
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
    
    subplot_df = (
        Z_true
        .assign(protected=lambda x: x['protected'].map({1: 'B', 0: 'A'}))
        #.assign(sex=lambda x: x['sex'].map({1: 'male', 0: 'female'}))
        .assign(y_pred=y_pred)
        .assign(y_true=y_true)
    )
    _subplot(subplot_df, 'protected', ax=axes[0])
    _subplot(subplot_df, 'y_true', ax=axes[1])
    _performance_text(fig, y_true, Z_true, y_pred, Z_pred, epoch)
    fig.tight_layout()
    

def _subplot(subplot_df, col, ax):
    for label, df in subplot_df.groupby(col):
        sns.kdeplot(df['y_pred'], ax=ax, label=label, shade=True)
    ax.set_title(f'Sensitive attribute: {col}')
    ax.set_xlim(0,1)
    ax.set_ylim(0,7)
    ax.set_yticks([])
    ax.set_ylabel('Prediction distribution')
    ax.set_xlabel(r'$P(target)$'.format(col))

def _performance_text(fig, y_test, Z_test, y_pred, Z_pred=None,
                     epoch=None):   

    if epoch is not None:
        fig.text(1.0, 0.9, f"Training epoch #{epoch}", fontsize='16')

    clf_roc_auc = metrics.roc_auc_score(y_test, y_pred)
    clf_accuracy = metrics.accuracy_score(y_test, y_pred > 0.5) * 100
    p_rules = {'protected': p_rule(y_pred, Z_test['protected'])}
    fp_rates = fp_rate(y_test, y_pred, Z_test['protected'])
    fn_rates = fn_rate(y_test, y_pred, Z_test['protected'])
    fig.text(1.0, 0.65, '\n'.join(["Classifier performance:",
                                   f"- ROC AUC: {clf_roc_auc:.2f}",
                                   f"- Accuracy: {clf_accuracy:.1f}"]),
             fontsize='16')
    fig.text(0.35, 0.6, '\n'.join(["FP-rates:"] +
                                 [f"- {attr}: {100*fp_rates[attr]:.2f}%" 
                                  for attr in fp_rates.keys()]), 
             fontsize='16')
    fig.text(0.35, 0.4, '\n'.join(["FN-rates:"] +
                                 [f"- {attr}: {100*fn_rates[attr]:.2f}%" 
                                  for attr in fn_rates.keys()]), 
             fontsize='16')
    fig.text(1.0, 0.4, '\n'.join(["Satisfied p%-rules:"] +
                                 [f"- {attr}: {p_rules[attr]:.0f}%-rule" 
                                  for attr in p_rules.keys()]), 
             fontsize='16')
    if Z_pred is not None:
        adv_roc_auc = metrics.roc_auc_score(Z_test, Z_pred)
        fig.text(1.0, 0.20, '\n'.join(["Adversarial performance:",
                               f"- ROC AUC: {adv_roc_auc:.2f}"]),
                                 fontsize='16')

In [ ]:
# load ICU data set
X, y, Z = load_GEN_data('data/generated.data')

In [ ]:
# split into train/test set
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, y, Z, test_size=0.5, 
                                                                     stratify=y, random_state=7)

# standardize the data
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler) 
X_test = X_test.pipe(scale_df, scaler) 

In [ ]:
class GenDataset(TensorDataset):

    def __init__(self, X, y, Z):
        X = torch.from_numpy(X.as_matrix()).float()
        y = torch.from_numpy(y.to_frame('y').as_matrix()).float()
        Z = torch.from_numpy(Z.as_matrix()).float()
        super(GenDataset, self).__init__(X, y, Z)
        self.X = X
        self.y = y
        self.Z = Z

In [ ]:
train_data = GenDataset(X_train, y_train, Z_train)
test_data = GenDataset(X_test, y_test, Z_test)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
class Classifier(nn.Module):
    
    def __init__(self, n_features, n_hidden=32, p_dropout=0.2):
        super(Classifier, self).__init__()
        self.dropout = nn.Dropout(p_dropout)
        self.out = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            self.dropout,
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            self.dropout,
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            self.dropout,
            nn.Linear(n_hidden, 1),
        )

    def forward(self, x):
        return F.sigmoid(self.out(x))
    

def pretrain_classifier(clf, data_loader, optimizer, criterion):
    for x, y, _ in data_loader:
        clf.zero_grad()
        p_y = clf(x)
        loss = criterion(p_y, y)
        loss.backward()
        optimizer.step()
    return clf

In [ ]:
clf = Classifier(X_train.shape[1])
clf_criterion = nn.BCELoss()
clf_optimizer = optim.Adam(clf.parameters())

In [ ]:
N_CLF_EPOCHS = 50

for epoch in range(N_CLF_EPOCHS):
    clf = pretrain_classifier(clf, train_loader, clf_optimizer, clf_criterion)

In [ ]:
with torch.no_grad():
    pre_clf_test = clf(test_data.X)

In [ ]:
y_pre_clf = pd.Series(pre_clf_test.data.numpy().ravel(),
                      index=y_test.index)
_ = plot_dists(y_test, Z_test, y_pre_clf)

In [ ]:
class Adverserial(nn.Module):
    
    def __init__(self, n_sensitive, n_hidden=32):
        super(Adverserial, self).__init__()
        self.out = nn.Sequential(
            nn.Linear(1, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_sensitive),
        )
        
    def forward(self, x):
        return F.sigmoid(self.out(x))

In [ ]:
def pretrain_adverserial(adv, clf, data_loader, optimizer, criterion):
    for x, _, z in data_loader:
        p_y = clf(x).detach()
        adv.zero_grad()
        p_z = adv(p_y)
        loss = criterion(p_z, z)
        loss.backward()
        optimizer.step()
    return adv

In [ ]:
adv = Adverserial(Z_train.shape[1])
adv_criterion = nn.BCELoss()
adv_optimizer = optim.Adam(adv.parameters())

In [ ]:
N_ADV_EPOCHS = 5

for epoch in range(N_ADV_EPOCHS):
    pretrain_adverserial(adv, clf, train_loader, adv_optimizer, adv_criterion)

In [ ]:
with torch.no_grad():
    pre_adv_test = adv(pre_clf_test)
    
y_pre_adv = pd.DataFrame(pre_adv_test.numpy(), columns=Z.columns)

In [ ]:
_ = plot_dists(y_test, Z_test, y_pre_clf, y_pre_adv)

In [ ]:
clf_adv_criterion = nn.BCELoss(reduce=False)

In [ ]:
lambdas = torch.Tensor([20])

In [ ]:
def train(clf, adv, data_loader, clf_criterion, adv_criterion,
          clf_adv_criterion, clf_optimizer, adv_optimizer):
    
    for x, y, z in data_loader:
        p_y = clf(x)

        # Train adversarial
        adv.zero_grad()
        p_adv = adv(p_y)
        loss_adv = adv_criterion(p_adv, z)
        loss_adv.backward(retain_graph=True)
        adv_optimizer.step()
        
    # Train classifier on last batch
    clf.zero_grad()
    clf_loss = clf_criterion(p_y, y) - (clf_adv_criterion(adv(p_y), z) * lambdas).sum()
    clf_loss.backward()
    clf_optimizer.step()
    
    return clf, adv

In [ ]:
N_EPOCH_COMBINED = 100

for epoch in range(N_EPOCH_COMBINED):
    
    clf, adv = train(clf, adv, train_loader, clf_criterion, adv_criterion,
                     clf_adv_criterion, clf_optimizer, adv_optimizer)
        
    with torch.no_grad():
        clf_pred = clf(test_data.X)
        adv_pred = adv(clf_pred)
    
    y_post_clf = pd.Series(clf_pred.numpy().ravel(), index=y_test.index)
    Z_post_adv = pd.DataFrame(adv_pred.numpy(), columns=Z_test.columns)
    
    if epoch % (N_EPOCH_COMBINED/10) == 0:
        plot_dists(y_test, Z_test, y_post_clf, Z_post_adv, epoch)